# Rango Consumos por Persona y Diagnósticos

Se calcula con consumos por período de prestación en un reporte y stock en otro reporte

In [21]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import datetime
import funciones

In [44]:
# DEFINICION DE LAS RUTAS Y VARIABLES:

# Archivo al que se exportara la informacion:
archivo_diagnosticos = 'C:/Users/lsaucedo/downloads/diagnosticos.csv'

# Rutas donde se encuentran los 12 meses de consumo y stock por persona:
ruta_cons_RC = "C:/Users/lsaucedo/Desktop/Costo PMO 202112/Consumo_Rango_Cons/"
ruta_stock_RC = "C:/Users/lsaucedo/Desktop/Costo PMO 202112/Stock_Rango_Cons/"


# Ruta con los 12 meses de stock de SanCor por persona:
ruta_stock_ss = "C:/Users/lsaucedo/Desktop/Costo PMO 202112/Stock_Sancor/"


# Defino importe gran consumidor
importe_gc = 703354
# De cada rango se define su límite inferior para consumo mensual
c_muy_alto = importe_gc / 12
c_alto = 0.015714129 * importe_gc # el monto se actualiza de acuerdo al importe gran consumidor
c_medio = 0.000785706 * importe_gc # el monto se actualiza de acuerdo al importe gran consumidor
c_bajo = 0.000314283 * importe_gc # el monto se actualiza de acuerdo al importe gran consumidor
# El consumo c_muy_bajo queda por defecto.


# proporcion limite para ser considerado persona con diagnostico
prop_limite = 0.4 



In [23]:
# Importamos los datos de consumo por periodo consumo y stock por periodo
dfc_RC = funciones.leer_xlsx(ruta_cons_RC)
dfs_RC = funciones.leer_xlsx(ruta_stock_RC)

# 

Inicio del proceso: 11:28:17
Se cargó el archivo:  30.1 Stock 12 M.xlsx a las: 11:28:42
Fin de la carga de archivos
Inicio del proceso: 11:28:42
Se cargó el archivo:  30.2 Consumo 12 M.xlsx a las: 11:29:06
Fin de la carga de archivos


In [24]:
# renombramos las columnas de ambos dataframes
funciones.renombrar_columnas(dfs_RC)
funciones.renombrar_columnas(dfc_RC)

In [30]:
# Agregamos a dfs el consumo de dfc:
dfs_RC = pd.merge(left = dfs_RC, right = dfc_RC, left_on='Persona', right_on='Persona', how = "left")
# Eliminamos el dfc de la memoria:
del dfc_RC

In [31]:
# Marco los GC en el dataframe de stock
dfs_RC['GC'] = np.where(dfs_RC['Consumo'] > importe_gc, 'GC','NO GC')

# Marco Rango de Consumos:
# Cálculo de consumo anualizado para rango consumos:
dfs_RC['Cons_mensualizado'] = (dfs_RC['Consumo'] / dfs_RC['Stock'])

# Como existen personas con stock == 0 -> reemplazo los nan con el valor de la columna "Consumo"
# No encontré una función -> hago una reimputación manual
for i in range(len(dfs_RC)):
    if np.isnan(dfs_RC["Cons_mensualizado"][i]) == True:
        dfs_RC["Cons_mensualizado"][i] = dfs_RC["Consumo"][i]

condiciones = [
    (dfs_RC['Cons_mensualizado'] >= c_muy_alto) ,
    (dfs_RC['Cons_mensualizado'] >= c_alto) ,
    (dfs_RC['Cons_mensualizado'] >= c_medio) ,
    (dfs_RC['Cons_mensualizado'] >= c_bajo) ,
    (dfs_RC['Cons_mensualizado'] < c_bajo)]

# Lista de resultados en función de las selecciones
valores_condic = ['Muy Alto', 'Alto', 'Medio', 'Bajo', 'Muy Bajo']     

# Generación de la columna calculada
dfs_RC['Rango_Cons'] = np.select(condiciones, valores_condic, default='Muy Bajo') # se analizaron y surgen de consumo 3 == 0


C:\Users\lsaucedo\AppData\Local\Temp/ipykernel_4408/3801111749.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_RC["Cons_mensualizado"][i] = dfs_RC["Consumo"][i]


In [ ]:
# OPCIONAL - Exportar el dataframe a csv para tener las marcas a futuro
#dfs_RC.to_csv("Rango_cons.csv", encoding = 'latin1', decimal=',', index = False)

In [36]:
## IMPORTAMOS EL DATAFRAME CON TODO EL STOCK PARA MARCAR LOS DIAGNOSTICOS
dfs_ss = funciones.leer_xlsx(ruta_stock_ss)
funciones.renombrar_columnas(dfs_ss)


Inicio del proceso: 11:41:22
Se cargó el archivo:  20.1 Stock Mendoza.xlsx a las: 11:42:44
Fin de la carga de archivos


In [37]:
# Pegamos a dfs_ss el rango de consumo:
dfs_ss = pd.merge(left = dfs_ss, right = dfs_RC[['Persona', 'GC', 'Rango_Cons']], left_on='Persona', right_on='Persona', how = "left")

In [38]:
# Existen casos con nan a reemplazar (decision leo)
# Al ser personas encontradas en el stock y no encontradas en el consumo ponemos: No GC y Rango Cons == Muy Bajo
values = {"GC": "NO GC", "Rango_Cons": "Muy Bajo"}
dfs_ss = dfs_ss.fillna(value=values)

In [40]:
# Transformaciones necesarias para el calculo de los diagnosticos como metricas:
dfs_ss['GC'] = dfs_ss['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs_ss['Diabetes'] = dfs_ss['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs_ss['Hepatitis'] = dfs_ss['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs_ss['PMI'] = dfs_ss['PMI'].replace(['NO', 'SI'], [0, 1])
dfs_ss['Cronico'] = dfs_ss['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs_ss['Oncologico'] = dfs_ss['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs_ss['GC'] = dfs_ss['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs_ss['Discapacidad'] = dfs_ss['Discapacidad'].replace(['N', 'S'], [0, 1])


# Creo una columna de resultado que marque 1 si se tiene cualquier diagnostico
suma = (dfs_ss['Diabetes'] + dfs_ss['Hepatitis'] + dfs_ss['PMI'] + 
        dfs_ss['Cronico'] + dfs_ss['Oncologico'] + dfs_ss['GC'] + dfs_ss['Discapacidad'])

dfs_ss['algun_diag'] = np.where( suma > 0, 1, 0)

In [41]:
# Creo una auxiliar que resuma por persona el stock y cantidad de veces con diag.
# Agrupo por persona las diferentes métricas en otro df para luego cruzar con la base original
filas = ['Persona','Rango_Cons']
columnas = ['Stock','Diabetes', 'Hepatitis', 'PMI','Cronico', 'Oncologico', 'GC', 'Discapacidad', 'algun_diag']
aux_diag = pd.DataFrame(dfs_ss.pivot_table(columnas , filas, aggfunc = np.sum))

# Genero una columna que calcule el % de veces que se tuvo algún diagnóstico:
aux_diag['prop_diag'] = aux_diag['algun_diag'] / aux_diag['Stock']
aux_diag.tail()

# Creamos la marca a cada persona
# Analizando las diferentes posibilidades llegué a la conclusión que si la proporción es mayor a 40% se debe poner "Con Diagnóstico"
aux_diag['Marca_Diagnostico'] = np.where(aux_diag['prop_diag'] > prop_limite, 'Con Diagnostico','Sin Diagnostico')

# Reseteo los multi index:
aux_diag = aux_diag.reset_index()

In [42]:
# Observo los primeros 8 registros
aux_diag.head(8)

,Persona,Rango_Cons,Cronico,Diabetes,Discapacidad,GC,Hepatitis,Oncologico,PMI,Stock,algun_diag,prop_diag,Marca_Diagnostico
0,14683,Medio,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico
1,21372,Medio,0,12,0,0,0,0,0,12,12,1.0,Con Diagnostico
2,30921,Medio,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico
3,49672,Alto,10,0,0,0,0,0,0,10,10,1.0,Con Diagnostico
4,54021,Muy Alto,0,0,0,12,0,0,0,12,12,1.0,Con Diagnostico
5,58897,Bajo,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico
6,90727,Medio,0,0,0,0,0,0,0,11,0,0.0,Sin Diagnostico
7,100900,Medio,0,0,0,0,0,0,0,12,0,0.0,Sin Diagnostico


In [ ]:
# Exporto el df a csv
aux_diag.to_csv(archivo_diagnostico, encoding = 'latin1',  decimal=',')

# 3- Calcular marcas de diagnóstico -- NO SE REALIZA MAS AQUI

El procedimiento a seguir debe ser el siguiente:<br>
1- Si el 100% del tiempo tuvo un diagnóstico va ese. <br>
2- Si más del 60% tuvo algún diagnóstico va ese.<br>
3- Si no supera 60% en algún clúster se lo define como su peor situación (con diagnóstico).<br>
4- (solamente para consumos) = si no se encuentra la persona en el tiempo de stock analizado se lo marca como "No Corresponde".<br>

In [ ]:
# dfs2 es el 2do dataframe que usamos para hacer el pegado de diag.
ruta_dfs2 = 'C:\\Users\\lsaucedo\\Desktop\\Brinkmann v2\\Stock_Brinkmann/'
dfs2 = pd.read_excel(ruta_dfs2 + "Stock_brinkmann_202010-202109.xlsx")

In [ ]:
dfs2.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE',
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3'                      
                           }, inplace=True)

In [ ]:
# Ruta de auxiliares
ruta_aux = 'C:\\modelos\\Auxiliares\\'

In [ ]:
# Definiciones:
# Proporción mínima de meses con diagnóstico para ser considerado con algún diagnóstico:
prop_limite = 0.4
# Se puede agregar los campos considerados como diag..

# Reemplazo valores de filas con replace:
dfs2['Diabetes'] = dfs2['Diabetes'].replace(['No Diabético', 'Diabético'], [0, 1])
dfs2['Hepatitis'] = dfs2['Hepatitis'].replace(['Sin Hepatitis', 'Con Hepatitis'], [0, 1])
dfs2['PMI'] = dfs2['PMI'].replace(['NO', 'SI'], [0, 1])
dfs2['Cronico'] = dfs2['Cronico'].replace(['NO', 'SI'], [0, 1])
dfs2['Oncologico'] = dfs2['Oncologico'].replace(['Sin Marca', 'Paciente Oncológico'], [0, 1])
dfs2['GC'] = dfs2['GC'].replace(['NO GC', 'GC'], [0, 1])
dfs2['Discapacidad'] = dfs2['Discapacidad'].replace(['N', 'S'], [0, 1])

# Creo una columna de resultado que marque 1 si se tiene cualquier diagnostico
# Se tendría que mejorar para generalizar:
suma = (dfs2['Diabetes'] + dfs2['Hepatitis'] + dfs2['PMI'] + 
        dfs2['Cronico'] + dfs2['Oncologico'] + dfs2['GC'] + dfs2['Discapacidad'])
dfs2['algun_diag'] = np.where(suma > 0, 1, 0)

# Creo una auxiliar que resuma por persona el stock y cantidad de veces con diag.
# Agrupo por persona las diferentes métricas en otro df para luego cruzar con la base original
filas = ['Persona','Rango_Cons']
columnas = ['Stock','Diabetes', 'Hepatitis', 'PMI','Cronico', 'Oncologico', 'GC', 'Discapacidad', 'algun_diag']
aux_diag = pd.DataFrame(dfs2.pivot_table(columnas , filas, aggfunc = np.sum))


# Genero una columna que calcule el % de veces que se tuvo algún diagnóstico:
aux_diag['prop_diag'] = aux_diag['algun_diag'] / aux_diag['Stock']
aux_diag.tail()

# Creamos la marca a cada persona
# Analizando las diferentes posibilidades llegué a la conclusión que si la proporción es mayor a 40% se debe poner "Con Diagnóstico"
aux_diag['Marca_Diagnostico'] = np.where(aux_diag['prop_diag'] > prop_limite, 'Con Diagnostico','Sin Diagnostico')

# Reseteo los multi index:
aux_diag = aux_diag.reset_index()

In [ ]:
# tener en cuenta que al agregar la pos. asoc. a la td pueden aparecer
#personas duplicadas si se exportan los reportes en diferente momento..

In [ ]:
# lo que se acorta el df tiene relacion con personas duplicadas.
# en el caso de stock continúa ok.
len(aux_diag)

In [ ]:
aux_diag.Stock.sum()

# Exporto la info a csv

In [45]:
aux_diag.to_csv(archivo_diagnosticos, encoding = 'latin1', decimal=',', index = False)